# Power-tracing smartcards using LEIA

In [ ]:
from pyecsca.sca.target.ectester import ECTesterTargetLEIA, KeypairEnum, ParameterEnum, CurveEnum, KeyEnum, KeyClassEnum, KeyBuildEnum, KeyAgreementEnum, SignatureEnum, TransformationEnum
from pyecsca.ec.params import load_params_ectester
from pyecsca.sca.scope.picoscope_sdk import PS6000Scope
from pyecsca.sca.trace import Trace
from pyecsca.sca.trace.plot import plot_trace, plot_traces
from pyecsca.sca.scope import SampleType

import numpy as np
from time import sleep
from smartleia import LEIA, TriggerPoints

import holoviews as hv

hv.extension("bokeh")
%opts RGB [height=700, responsive=True]

In [ ]:
sl = LEIA()
ectester = ECTesterTargetLEIA(sl)

In [ ]:
ectester.connect()

In [ ]:
atr = ectester.atr
print(atr, atr.hex())
card_map = {
    "3bd518ff8191fe1fc38073c8211309": "A1",
    "3bb89600c00831fe45ffff1154305023006a": "I1",
    "3bfe1800008031fe45803180664090a5102e1083019000f2": "I2",
    "3bf81800ff8131fe454a434f507632343143": "N1",
    "3bf81300008131fe454a434f5076323431b7": "N2N9",
    "3b9495810146545601c4": "N4",
    "3bd518ff8191fe1fc38073c821100a": "N6",
    "3b9c9580811f039067464a01005404f272fe00c0": "F1",
    "3b90958011fe6a": "F2",
    "3b9f95803fc7a08031e073fa21106300000083f09000bb": "S1S2"
}
card = card_map.get(atr.hex(), None)
print(card)

In [ ]:
scope = PS6000Scope()

In [ ]:
scope.open()

In [ ]:
print(scope.get_variant())
if card == "A1":
    # Athena IDProtect
    # 35M for keygen
    # 13M for ecdh
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=35_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.24, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "I1":
    # Infineon SECORA
    # 6M for keygen
    # 3M for ecdh
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=6_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.25, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "I2":
    # Infineon CJTOP SLJ 52GLA0890AL M84
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=15_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.1, offset=-0.15, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "N1":
    # NXP J3A081
    # 30M for keygen (first), then 10M for subsequent
    # 10M for ecdh
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=30_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.05, offset=-0.18, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "N2N9":
    # NXP JCOP v2.4.1R3
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=30_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.05, offset=-0.280, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "N4":
    # NXP J3H145
    # 15M for keygen
    # 10M for ecdh
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=5_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.160, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "N6":
    # NXP JCOP4
    # 3M for keygen
    # 3M for ECDH
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=3_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.170, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "F1":
    # Javacos A22 CR
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=3_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.170, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "F2":
    # Javacos JC30M48 CR
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=3_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.170, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
print(actual_freq, n_samples)

In [ ]:
ectester.select_applet()

In [ ]:
ectester.info()

In [ ]:
ectester.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [ ]:
ectester.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH)

In [ ]:
params = load_params_ectester("curves/curves_full_order/cofactor256p313_full.csv", "affine")

In [ ]:
ectester.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetLEIA.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
sl.set_trigger_strategy(1, point_list=[TriggerPoints.TRIG_PRE_SEND_APDU], delay=0)

In [ ]:
scope.arm()
sleep(2)

In [ ]:
ectester.generate(KeypairEnum.KEYPAIR_LOCAL)

In [ ]:
scope.capture(10000)

In [ ]:
trace_gen = scope.retrieve("B", SampleType.Raw)

In [ ]:
plot_trace(trace_gen)

In [ ]:
scope.arm()
sleep(2)

In [ ]:
with open("curves/cofactor_points/point_313.csv", "r") as f:
    line = f.read()
    sx, sy = line.split(",")
    bx = bytes.fromhex(sx[2:])
    by = bytes.fromhex(sy[2:])
    point = bytes([0x04]) + bx + by
    print(point.hex())

In [ ]:
ectester.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                     True,
                     TransformationEnum.NONE,
                     KeyAgreementEnum.ALG_EC_SVDP_DH,
                     point) # pubkey as bytes

In [ ]:
scope.capture(10000)

In [ ]:
trace_ecdh = scope.retrieve("B", SampleType.Volt)

In [ ]:
plot_trace(trace_ecdh)

In [ ]:
from pyecsca.sca.trace.edit import pad, trim

In [ ]:
trim?

In [ ]:
plot_traces(ecdh_ok, pad(trim(trace_ecdh, 0, len(trace_ecdh) -363700), (363700, 0)))

In [ ]:
from scipy import signal
import numpy as np

In [ ]:
corr = signal.correlate(trace_ecdh.samples, trace_ecdh.samples, mode="full")
lags = signal.correlation_lags(trace_ecdh.samples.size, trace_ecdh.samples.size, mode="full")
lag = lags[np.argmax(corr)]


In [ ]:
plot_trace(Trace(corr))

In [ ]:
ectester.cleanup()

In [ ]:
ectester.disconnect()

In [ ]:
scope.close()

In [ ]:
from pyecsca.sca.trace.process import rolling_mean, recenter
from pyecsca.sca.trace.filter import filter_lowpass

In [ ]:
from pyecsca.sca.trace.plot import plot_trace, plot_traces